# Credit Repayment Behaviour Analysis

The goal to find out whether the factors such as the marital status and the number of children of the borrowers affect the repayment of the loan to the bank on time. The Bank X has provided the data on clients' creditworthiness.

The report will be considered when building a credit scoring of a potential client of the bank. A credit scoring is used to evaluate the risks of the potential borrowers defaulting on their debt obligations.

# Data Description

***Note:*** the data provided by the Bank X is about clients in Russia, thus some contents of the dataset might be in Russian language.

children : the number of children in the family
<br>days_employed: how long the customer has worked
<br>dob_years: the customer’s age
<br>education: the customer’s education level
<br>education_id: identifier for the customer’s education
<br>family_status: the customer’s marital status
<br>family_status_id: identifier for the customer’s marital status
<br>gender: the customer’s gender
<br>income_type: the customer’s income type
<br>debt: whether the client has ever defaulted on a loan
<br>total_income: monthly income
<br>purpose: reason for taking out a loan

## Step 1. Data quality assessment

In [445]:
# importing libraries
import pandas as pd 
from pymystem3 import Mystem # getting a stemmer/lemmatizer for words in Russian
import collections
import nltk
from nltk.stem.snowball import SnowballStemmer
russian_stemmer = SnowballStemmer('russian')
from nltk.stem import WordNetLemmatizer
wordnet_lemma = WordNetLemmatizer()
m = Mystem()

In [446]:
# opening the data file
df = pd.read_csv('clients_data.csv')

In [447]:
# getting the first 5 table strings.
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [448]:
# looking at the general information
df.info()
# checking numeric values
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [449]:
# checking for at the NaN values
df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**Summary:** We have a table with the total of 21525 rows and 12 columns of clients' data. Data types include float64, int64, and object. 
Findings:

1. 'total_income', 'days_employed' each have 2174 NaN values. 
2. 'days_employed': has negative and very big values, such as '-18388.949901' and '401755.400475'. The data in this column doesn't make sense as it is meant to represent the number of days of the employment. It is necessary to request more information about this data.
3. 'children': the number of children '-1' and '20'
4. 'dob_years': some of the client's data indicate their age as '0' years
5. 'education', 'education_id': there are 15 unique values in 'education' and 5 in 'education_id'
6. 'gender': 3 unique values: female, male, and N/A

Possible reasons for missing values & errors in the data above: In this case, it might be the concealment of the personal information by the client (about the employment and income level). Another possibility is the data entry errors.

## Step 2. Data preprocessing

### Processing the data with errors

Now we need to process the column 'children' where we saw such improbable values as -1 and 20 (number of children):

In [450]:
# check the values
print(df['children'].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


Following the logic that 20 and -1 were data entry errors, we can replace them assuming that 20 is an error is for 2 children, and -1 is an error for 0 

In [451]:
df['children'] = df['children'].mask(df['children']==20, 2) 
df['children'] = df['children'].mask(df['children']==(-1), 0) 

    # check 
print(df['children'].value_counts())

0    14196
1     4818
2     2131
3      330
4       41
5        9
Name: children, dtype: int64


**Summary:** We have processed the data in 'children' column and now the minimum number of children is 0 and maximum is 5, which seems reasonable

### Processing the missing values

To process the missing values in the data without affecting its accuracy, we need to divide data in groups (where possible) and fill in the missing values within each group.

For example, 'total_income' has 2174 missing values but we also know that we have a calumn 'income_types' (with such values as 'employee', 'retired') that could help us identify their level of income, so we will group 'total_income' by 'income_types' and will find a median value in each group.

In [452]:
# grouping 'total_income' by 'income_type'
income_dict = dict(df.groupby('income_type')['total_income'].median())

In [453]:
# replacing the missing values in 'total_income' column with values from 'income_dict'
df['total_income'] = df['total_income'].fillna(df['income_type'].apply(lambda x: income_dict.get(x)))

In [454]:

# filling in the missing values in 'days_employed' column with median, but without the dictionary
# because number of days of employment does not depend on the other parameters in the dataset
df['days_employed'] = df['days_employed'].fillna(df['days_employed'].median())
     #Check:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Summary:** We've processed the missing values and can move on to the next step

### Data type replacement

In [455]:
# replacing data types for the total_income', 'days_employed' with .astype() method
df[['days_employed', 'total_income']] = df[['days_employed', 'total_income']].astype('int')
    # Check
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int32 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int32 
 11  purpose           21525 non-null  object
dtypes: int32(2), int64(5), object(5)
memory usage: 1.8+ MB


**Summary:** We've replaced the 'float64' data type with 'int' for convenience and clarity

### Processing the data duplicates

In [456]:
# checking the values in 'education'
df['education'].value_counts() 

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

* **Eng translation** 
<br>secondary education    13750
<br>masters degree          4718
<br>SECONDARY EDUCATION      772
<br>Secondary Education      711
<br>bachelor degree          668
<br>MASTERS DEGREE           274
<br>Masters Degree           268
<br>primary education        250
<br>Bachelor Degree           47
<br>BACHELOR DEGREE           29
<br>PRIMARY EDUCATION         17
<br>Primary Education         15
<br>academic degree            4
<br>ACADEMIC DEGREE            1
<br> Academic Degree            1

We can see that we have duplicates in this column that need to be processed.

In [457]:
# using  str.lower() method
df['education'] = df['education'].str.lower()

    # Check
df['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

* **Eng translation**
<br>secondary education    15233
<br>masters degree          5260
<br>bachelor degree          744
<br>primary education        282
<br>academic degree            6

**Summary:** by bringing the data in 'education' to lowercase, we now have 'education' with 'education_id' in the same row and duplicates are sorted

In [458]:
# counting teh remaining number of duplicates in the dataframe
print('Remaining duplicates:',df.duplicated().sum())

Remaining duplicates: 71


In [459]:
# dropping duplicates + check
df = df.drop_duplicates().reset_index(drop= True)
print('Remaining duplicates after dropping:',df.duplicated().sum())

Remaining duplicates after dropping: 0


**Summary:** We've dropped all the data duplicates.

### Stemming/Lemmatization

We need to stem (or lemmatize) a 'purpose' column. To see which method is better, let's check its unique values first:

In [460]:
# using .value_couns()
df['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

* **Eng translation:**
<p>wedding ceremony                                 791<br>
having a wedding                                 768<br>
to have a wedding                                765<br>
real estate transactions                         675<br>
buy commercial real estate                       661<br>
housing transactions                             652<br>
buying property for renting out                  651<br>
transactions with the residential real estate    650<br>
purchase of the house                            646<br>
housing                                          646<br>
purchase of the house for my family              638<br>
construction of own property                     635<br>
property                                         633<br>
transactions with my real estate                 627<br>
building a real estate                           624<br>
buy real estate                                  621<br>
purchase of my own house                         620<br>
building a property                              619<br>
property renovation                              607<br>
buy residential real estate                      606<br>
buying my own car                                505<br>
going to university                              496<br>
car                                              494<br>
second-hand car purchase                         486<br>
cars                                             478<br>
buying a second-hand car                         478<br>
to own a car                                     478<br>
to buy a car                                     471<br>
car purchase                                     461<br>
supplementary education                          460<br>
purchase of a car                                455<br>
university education                             452<br>
education                                        447<br>
to get asupplementary education                  446<br>
getting an education                             442<br>
profile education                                436<br>
getting higher education                         426<br>
to become educated                               408


Find a stem the 'purpose' column:

In [461]:
def stem_text(text):
    words = text.split() # split text
    new_text = ' '.join([russian_stemmer.stem(w) for w in words])
    return new_text

df['purpose'] = df['purpose'].apply(stem_text)

# check the values
df['purpose'].value_counts() 

свадьб                            791
на проведен свадьб                768
сыгра свадьб                      765
операц с недвижим                 675
покупк коммерческ недвижим        661
операц с жил                      652
покупк жил для сдач               651
операц с коммерческ недвижим      650
покупк жил                        646
жил                               646
покупк жил для сем                638
строительств собствен недвижим    635
недвижим                          633
операц со сво недвижим            627
строительств жил недвижим         624
покупк недвижим                   621
покупк сво жил                    620
строительств недвижим             619
ремонт жил                        607
покупк жил недвижим               606
на покупк сво автомобил           505
заня высш образован               496
автомобил                         494
сделк с подержа автомобил         486
сво автомобил                     478
на покупк подержа автомобил       478
автомоб     

* **Eng translation:**
<p>car                                             972<br>
wedding ceremony                                791<br>
having a wedding                                768<br>
to have a wedding                               765<br>
real estate transaction                         675<br>
buy commercial real estate                      661<br>
housing transaction                             652<br>
buying property for renting out                 651<br>
transaction with the residential real estate    650<br>
purchase of the house                           646<br>
housing                                         646<br>
purchase of the house for my family             638<br>
construction of own property                    635<br>
property                                        633<br>
transaction with my real estate                 627<br>
building a real estate                          624<br>
buy real estate                                 621<br>
purchase of my own house                        620<br>
building a property                             619<br>
property renovation                             607<br>
buy residential real estate                     606<br>
buying my own car                               505<br>
going to university                             496<br>
second-hand car purchase                        486<br>
buying a second-hand car                        478<br>
to own a car                                    478<br>
to buy a car                                    471<br>
car purchase                                    461<br>
supplementary education                         460<br>
purchase of a car                               455<br>
university education                            452<br>
education                                       447<br>
to get asupplementary education                 446<br>
getting an education                            442<br>
profile education                               436<br>
getting higher education                        426<br>
to become educated                              408</p>

**Summary:** We've completed the stemming and now we can use it to group our data

### Categorizing Data

#### Categorising data in 'purpose'

We have identified 4 big groups in a 'purpose'. Now we will group the data by assigning 'автомобиль'("car"), 'свадьба'("wedding"), 'недвижимость'("real estate"), 'образование'("education"):

In [462]:
def lemma_group(text):
    if russian_stemmer.stem('автомобиль') in text:
        return 'автомобиль'
    if russian_stemmer.stem('свадьба') in text:
        return 'свадьба'
    if russian_stemmer.stem('недвижимость') in text or russian_stemmer.stem('имущество') in text or russian_stemmer.stem('дом') in text or russian_stemmer.stem('жильё') in text:
        return 'недвижимость'
    if russian_stemmer.stem('университет') in text or russian_stemmer.stem('образование') in text:
        return 'образование'

df['purpose'] = df['purpose'].apply(lemma_group)

    # check
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба


Which is the most popular 'purpose' of credit in the bank?

In [463]:
df['purpose'].value_counts()

недвижимость    10811
образование      4013
автомобиль       3828
свадьба          2324
Name: purpose, dtype: int64

* **Eng translation:**
<p>real estate    10811<br>
car             4306<br>
education       4013<br>
wedding         2324</p>

As we see, the most popular reason to take a credit is a 'real estate' with a total of 10811 number of values. This seems like a reasonable answer as the cost of the real estat is very high and is unafordable for majority of the population. The least popular reason for taking a bank credit is 'wedding', with only 2324 values.

#### Categorizing the data in 'family_status'

In [475]:
# check the values
df['family_status'].value_counts() 

женат / замужем          12339
гражданский брак          4151
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

* **Eng translation**
<p>family_status	family_status_id<br>
0	married	12339<br>
1	civil partnership	4151<br>
2	widow / widower	2810<br>
3	divorced	1195<br>
4	unmarried	959</p>

As we can notice, the results for 'family_status' and 'family_status_id' represent the same thing, so we can group them together:

In [464]:
# creating the dictionary for 'family_status', 'family_status_id', droping the duplicates
family_dict = df[['family_status', 'family_status_id']].drop_duplicates().reset_index(drop=True)
display(family_dict)

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


* **Eng translation**
<p>family_status	family_status_id<br>
0	married	0<br>
1	civil partnership	1<br>
2	widow / widower	2<br>
3	divorced	3<br>
4	unmarried	4</p>

Now we can safely drop 'family_status_id' column.

In [465]:
# dropping 'family_status_id' column
df.drop(columns=['family_status_id'],inplace=True)

# check the results
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   children       21454 non-null  int64 
 1   days_employed  21454 non-null  int32 
 2   dob_years      21454 non-null  int64 
 3   education      21454 non-null  object
 4   education_id   21454 non-null  int64 
 5   family_status  21454 non-null  object
 6   gender         21454 non-null  object
 7   income_type    21454 non-null  object
 8   debt           21454 non-null  int64 
 9   total_income   21454 non-null  int32 
 10  purpose        20976 non-null  object
dtypes: int32(2), int64(4), object(5)
memory usage: 1.6+ MB


#### Categorizing the data in 'total_income'

In [466]:
# using .describe() method to know min, 25 %, mean, 75 % and max income
df['total_income'].describe() 

count    2.145400e+04
mean     1.653196e+05
std      9.818730e+04
min      2.066700e+04
25%      1.076230e+05
50%      1.425940e+05
75%      1.958202e+05
max      2.265604e+06
Name: total_income, dtype: float64

Divide the data by 4 main levels of income: 'низкий'(as for 'low'), 'средний' (as for 'middle'), 'выше среднего' (as for 'high'), 'высокий' (as for 'very high')

In [467]:
def income_level(income):
    if income < 1.076522e+05:
        return 'низкий'
    if income < 1.518870e+05:
        return 'средний'
    if income < 2.024170e+05:
        return 'выше среднего'
    else:
        return 'высокий'

# using .apply() method to process column
df['income_level'] = df['total_income'].apply(income_level) 

# check the values
df['income_level'].value_counts() 

средний          6574
низкий           5368
высокий          4909
выше среднего    4603
Name: income_level, dtype: int64

* **Eng translation**
<p>middle       5870<br>
low     5359<br>
high    5317<br>
very high       4908</p>

We divided people by income and now its clear that the clients with the income tipe of 'middle' take the most credits in this bank

#### Categorizing the data in "debt"

In [468]:
# check the values
df['debt'].value_counts() 

0    19713
1     1741
Name: debt, dtype: int64

1 here means that the client has defaulted on a loan.<br>
0 means that the client repaid a loan.

Divide the data by 2 main categories: 'выплачен вовремя'(as for 'paid on time') and 'не выплачен' (as for 'defaulted')

In [469]:
def debt_type(debt):
    if debt == 0:
        return 'выплачен вовремя'
    if debt == 1:
        return 'не выплачен'

# use apply() method to process column
df['debt_type'] = df['debt'].apply(debt_type) 

# check the values
df['debt_type'].value_counts()

выплачен вовремя    19713
не выплачен          1741
Name: debt_type, dtype: int64

* **Eng translation**
<p>paid on time      19713<br>
defaulted     1741</p>

Let's count the difference between "paid on time" and "defaulted" (%)

In [470]:
percentage = 1741 / 19713 * 100
print(round(percentage,2),'%')

8.83 %


We've made a more visual representation of debtors and on time paid. The difference between "paid on time" and "defaulted" is 8.83 %. So the most of the clients successfully  repay their loan on time.

#### Categorizing the data in 'education'

In [471]:
 # check the values
df['education'].value_counts()

среднее                15172
высшее                  5250
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

* **Eng translation**
<p>secondary education    15172<br>
masters degree          5250<br>
bachelor degree          744<br>
primary education        282<br>
academic degree            6</p>

'secondary education' and 'masters degree' are the most popular groups

If we stand back and look, the results of 'education' and 'education_id' represent the samme thing
So we'll create dictionary for them, as we did for 'family_status_id'


In [472]:
# create the dictionary for 'education', 'education_id', drop duplicates
education_dict = df[['education', 'education_id']].drop_duplicates().reset_index(drop=True)
display(education_dict)

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


* **Eng translation**
<p>education	education_id<br>
0	masters degree	0<br>
1	secondary education	1<br>
2	bachelor degree	2<br>
3	primary education	3<br>
4	academic degree	4</p>

Now we can safely we can drop "education_id" column.

In [473]:
df.drop(columns=['education_id'],inplace=True)

**Summary:** 
1. The most popular purpose for credit is for a real estate (10811 vs ~2000-4000 for other purposes)
2.  Majority of bank clients who apply for credit are married (12339 vs 959-4000 in other categories) 
3. Majority of people pay their debt on time (19713 vs 1741)
4. Most people who apply for the credit in the bank have the secondary education (15172 vs 5250 on masters degree and 6-700 on other types)

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [474]:
#См. Markdown
#Создадим новую Pivot Table, которая отражает количество должников с разным кол-вом детей в числах и процентах

data_pivot_new = df.pivot_table(index = 'children',  
                                values = 'debt', aggfunc = ['count', 'sum', 'mean'])
data_pivot_new.columns=['Total entries', 'Total debtors', '% of debtors']
data_pivot_new.style.format({'% of debtors': '{:.1%}'})

ImportError: Missing optional dependency 'Jinja2'. DataFrame.style requires jinja2. Use pip or conda to install Jinja2.

**Вывод**

Не обязательно. Из таблицы понятно, что у большинства опрошенных (14 138 людей) детей нет, а должников среди них только 1064. Наибольший процент должников у категории людей с 4мя детьми(9.8% или 4 человека), но среди них был только 41 респодент. Интересно, что при этом у категории людей с 5 детьми (всего 9 человек) нет долгов. Из данных результатов я бы сделала вывод, что на % должников влияют не дети или не только дети, и важно рассмотреть также другие факторы. Тип займа в банке тоже имеет значение.

<div class="alert alert-danger">
<h2> Комментарий ревьюера <a class="tocSkip"> </h2>

Здесь ты посчитала абсолютные значения, однако они не в полной мере отражают характеристики этих групп. Возможно, что общее кол-во людей с детьми намного больше людей без детей, поэтому и число задолжников больше. Сразу и не скажешь. Тебе необходимо посчитать относительные показатели, т.е какой процент людей с детьми невозращают кредит относительно общего кол-ва людей этой группы (комментарий общий, относится ко всему 3 разделу). Поэтому я предлагаю выполнить следующее:
    
- построй сводные таблицы по созданным столбцам и посчитай в них средние значения по столбцу `debt`. Это тоже самое что и поделить кол-во должников (операция sum) на общее кол-во клиентов (count). Этот показатель как раз и будет демонстрировать процент должников этих групп. 
</div>

<div class="alert alert-success">
<h2> Комментарий ревьюера v2 <a class="tocSkip"> </h2>

Там ты считаешь процент пользователей определенной группы от общего кол-ва пользователей. Это немного не то, чтобы посчитать кол-во должников, можно выполнить следующее (Обрати внимание на [style](https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html) библиотеки `pandas`. Там много прикольных штук для того, чтобы сделать датафрейм красивее)

In [ ]:
df_example = df.pivot_table(index = 'children', values = 'debt', 
                            aggfunc = ['count', 'sum', 'mean', lambda x: 1 - x.mean()])
df_example.columns = ['Кол-во пользователей', 'Кол-во должников', '% должников', '% НЕдолжников']
df_example.style.format({'% должников': '{:.2%}', '% НЕдолжников': '{:.2%}'})

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
#См. Markdown

**Вывод**

Да. Исходя из PivotTable в шаге 2.5, и результатам TOTAL по строкам (то есть, не важно, сколько детей у человека, смотрим только на семейное положение), можно сделать вывод, что кол-во женатых / замужних с долгами больше, значительно больше, чем у всех остальных, однако у не женатых / не замужних долгов больше, чем у тех, кто в разводе, вдова/вдовец или в гражданском браке. 

|family_status|TOTAL |  
|---|---|
|Не женат / не замужем |  **274**|
|в разводе     |         **85**|
|вдовец / вдова     |     **63**|
|гражданский брак |       **388**|
|женат / замужем    |     **931**|

<div style="border:solid purple 5px; padding: 20px"> 
<h2 align="center"> Рубрика «Питонячий лайфхакер» <a class="tocSkip"> </h2>

<h3> Функция zip <a class="tocSkip"> </h3>

Функция zip создаёт итератор, который комбинирует элементы нескольких списков. Это позволяет осуществлять параллельный обход списков в циклах for или, например, выполнять параллельную сортировку.

![](https://i.ibb.co/MPPZ6TL/image.png)

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
#По официальным данным от Росстата, среднемесячная ЗП населения России в 2020 51083. 
#Для того чтобы считаться средним классом, в Москве необходимо зарабатывать от 121 тыс. руб

#Посчитаем, что средний ур. ЗП это приблизительно 51000-121000тыс руб. Ниже 51к - ниже ср., Выше 120к - выше ср.
def total_income_category(value):
    if value < 51000:
        return 0
    elif 51000 <= value <= 121000:
        return 1
    return 2 
df['income_level_index'] = df['total_income'].apply(total_income_category)

#Сделаем еще 1 PivoTable, разделив должников по гендерному признаку:
data_pivot = df.pivot_table(index=['income_level_index'], columns=['gender'], values='debt', aggfunc=['sum'],fill_value = 'N/A', margins = True, margins_name='TOTAL')
print(data_pivot)



<div class="alert alert-success">
<h2> Комментарий ревьюера <a class="tocSkip"> </h2>

Подкрепление выводов внешними фактами 👍. Усиливает ощущение хорошо сделанной работы

</div>

<div class="alert alert-warning">
<h2> Комментарий ревьюера <a class="tocSkip"> </h2>

Тебя не смутило значение `XNA` в данных?

<div class="alert alert-success">
<h2> Комментарий ревьюера <a class="tocSkip"> </h2>

Здесь, кстати для деления по доходу можно было применить метод [cut](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.cut.html) или [qcut](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.qcut.html)(в этом случае деление будет проводиться по проценилям). Обрати внимание, у данного метода есть параметр `labels`, так можно оформить группы красивее 😊

</div>

**Вывод**

Из Pivot Table выше можно сделать следующие выводы: 

1. Кол-во людей с ур. дохода выше среднего (индекс 2) гораздо больше тех, у кого ур. дохода ниже (1 и 0). Всего есть 6 должников с уровнем дохода ниже среднего (индекс 0)
                 
|income_level_index|TOTAL|                       
|---|---|
|0|**6**|
|1|**572**|
|2|**1143**|

2. Дополнительно, так как я отсортировала должников по гендерному признаку, можно сделать вывод, что кол-во должников женщин с доходом выше среднего, чем у мужчин с доходом выше среднего. То же самое относится к с реднему и ниже среднего уровням дохода. 

|income_level_index|Female|Male|XNA  
|---|---|---|---|                         
|0|18.0|8.0|N/A  
|1|386.0|186.0|N/A   
|2|590.0|553.0|0   
|**TOTAL**|**994.0**|**747.0**|**0**   


<div class="alert alert-success">
<h2> Комментарий ревьюера <a class="tocSkip"> </h2>

В Markdown редко таблицы визуализируют, видишь, в твоей реализации они получаются совсем нечитабельными. Почитай вот [здесь](https://medium.com/analytics-vidhya/the-ultimate-markdown-guide-for-jupyter-notebook-d5e5abf728fd), там есть примеры как сделать таблицы и в целом очень круто расписаны Markdown, и еще вот такую [шпаргалку](https://paulradzkov.com/2014/markdown_cheatsheet/) оставлю, вдруг понадобится 🙂
    
</div>

<font color='blue'>***Вероника***: Спасибо!</font> 

- Как разные цели кредита влияют на его возврат в срок?

In [ ]:
#Создадим еще 1 PivotTable
data_pivot = df.pivot_table(index=['group_purpose'], columns='gender', values='debt', aggfunc='sum',fill_value = 'N/A', margins = True, margins_name='TOTAL')
print(data_pivot[['F','M', 'TOTAL']])

**Вывод**

Из Pivot Table выше можно сделать следующие выводы: 
1. Больше всего должников среди двух имеющихся гендеров - те, кто берут кредит с целью покупки чего-либо (спецефически женщин должников больше, чем мужчин). На 2 месте среди целей для кредита - образование, на 3 - автомобиль. Во всех случаях кол-во должников женщин больше, чем мужчин.
group_purpose  TOTAL                        
автомобиль     277
жилье          46
недвижимость   42
образование    370
операция       205
покупка        436
ремонт         35
свадьба        186
строительство  144


## Шаг 4. Общий вывод

Разберем итоги на несколько этапов:

1)После общей оценки состояния датасетя, были выявлены несколько очевидных проблем: шибки в данных, разные регистры итд. 2. Возможные причины появления пропусков: чаще всего причиной является человеческий фактор. В данном случае, это может быть сокрытие личной информации (о занятости и уровне дохода). Другой возмодной причиной могут быть ошибки ввода данных.

2)Пропуски в столбце 'total_income' были заполнены медианой после разделения по группам по доходу('income_type'), т.к. есть среди данных есть неравномерное распределение среди небольших и крупных значений, что может повлять на точность анализа. Столбец 'days_employed' был заполнен этот столбец средним статистическим значением в пропусках, чтобы выровнять датасет. Из-за ошибки формате/вводе данных в данном столбце, он не использовался при дальнейем анализе.

3) Был заменен вещественынй тип данных - тип float - на целочисленный. Всего в данных 2 столбца, которые содержат entries данного типа: 'days_employed' (трудовой стаж в днях) и 'total_income' (доход в месяц). Заменяем значения на целочисленные для краткости и наглядности.

4)Для поиска и удаления дубликатов было использовано четыре метода: первый - найти уникальные значения в столбцах str, благодаря чему обнаружилось, что есть слова с одним значением, но разного регистра, которые считаются "уникальными"(это мешает поиску дубликатов), второй - исп. str.lower() метод для Pandas, чтобы привести строки в единый нижний регистр, третий - найти кол-во дубликатов двойным методом duplicated().sum(), четвертый - удалить дубликаты методом drop_duplicates.

Причин появления дубликатов может быть много, например: человеческий фактор (некорректное введение, ошибки при вводе данных, сокрытие информации, и т.д.) или технические ошибки, например неправильный свод данных их разных источников или баг

5)При просмотре данных о кол-ве детей у кадого респондента выяснилось, что всего в таблице пришлось 76 значений на 20 детей и 47 значений на -1, а все остальные распределены между 0-5 детей. Из этого я сделала вывод, что при вводе данных произошла ошибка, и те, кто написали 20 - дописали лишний 0, а при -1 подразумевалось 0. Спросить о достоверности данного вывода нет возможности, поэтому вместо удаления данных строк, я воспользовалась данной логикой при замене. В итоге, я собрала интересующие нас числа в PivotTable

***ИТОГ***: Изначальной поставленной задачей было выяснить влияет ли семейное положение и количество детей клиента на факт возврата кредита в срок. На анализе имеющихся данных можно сделать вывод, что наличие детей не обязательно является фактором влияния на просрочки платежей в банке,% должников в каждой группе разделенной по кол-ву детей примерно около 9% (за исключении группы с 5ю детьми). При этом, у женатых/ замужних без детей должников более чем в 2 раза больше, чем у любых других категорий. 

|family_status/children|0|1|2|3|4|5|  
|---|---|---|---|---|---|---|
|Не женат / не замужем|210.0|52.0|10.0 |1.0|1.0|N/A   |
|в разводе | 55.0 |21.0  |  8.0|   1.0 | 0.0|  N/A    |
|вдовец / вдова   |       53.0   | 7.0  |  3.0 |  0.0 | 0.0|  N/A|     
|гражданский брак   |     229.0 | 118.0  | 33.0|  8.0 | 0.0 |   0 |   
|женат / замужем   |      517.0  |246.0 | 148.0  |17.0|  3.0  |  0   |

Однако делать вывод, что не стоит выдавать кредит замужним / женатым женщинам или мужчинам если у них нет детей - неправильно. Нужен гораздо более глубокий анализ данных с учетом множества других факторов, которые могут повлиять на такие результаты. Например, у пары супругов нет детей, из-за проблем со здорвьем. 

**Рекоммендации**: 1) Уточнить формат данных в столбце 'days_employed' и перевести числа в корректную форму. 
2) необходим более повторный анализ данных с учетом множества других факторов, которые помогут ответить на вопрос влияет ли кол-во детей на наличие просрочек выплаты в банк. Конкретно, добавить в опрос несколько более подробных вопросов, прим.: 1) Есть ли в семье люди с инвалидностью/тяжелой болезнью? 2) Имеются ли долги в других банках/источниках? 3) Какие в среднем расходы на ребенка в месяц? итд

<div style="border:solid purple 2px; padding: 20px"> 

У тебя получилась очень хорошая работа! Многие моменты сделаны просто прекрасно. Ты показываешь отличное владение изучаемым материалом: уверенно пользуешься pandas, умеешь подготавливать, очищать, обогощать данные. Остались небольшие доработки:

- По всему проекту необходимо оставить и раскомментировать промежуточные решения
- В шаге 3 необходимо получить результаты с помощью сводных таблиц с расчетом относительных показателй по группам

Жду твоих исправлений, у тебя все получится 😊

</div>

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.